---
title: "Artificial Intelligence and Data Analytics"
subtitle: "Reinforcement Learning: Monte Carlo Tree Search"
author: "solar-san"
date-modified: "2023-09-26"
format:
  html:
    theme: github
    toc: true
    toc-location: left
    fig-align: center
    fig-width: 8
    fig-height: 8
    html-math-method: katex
    code-overflow: scroll
    code-copy: hover
    highlight-style: gruvbox
    citations-hover: true
    footnotes-hover: true
    header-includes: |
      <link rel="preconnect" href="https://fonts.googleapis.com">
      <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
      <link href="https://fonts.googleapis.com/css2?family=Atkinson+Hyperlegible:ital,wght@0,400;0,700;1,400;1,700&family=Fira+Code&display=swap" rel="stylesheet">
mainfont: "Atkinson Hyperlegible"
monofont: 'Fira Code'
---

![](header/AIDA-Project_header.png)

# Agent Carletto and the Tree Search

The following notebook contains all the code and a textual explanation of the Monte Carlo Tree Search algorithm implementation, aiming to obtain a challenging AI player, able to evaluate the state of the game and select the best move at each turn, having perfect knowledge of all the other players' cards.

What is a Monte Carlo Tree Search?

First of all, the term _Monte Carlo_ refers to a set of _methods_:

> Monte Carlo methods may be thought of as a collection of computational techniques for the (usually approximate) solution of mathematical problems, which make fundamental use of random samples.

The slides contain a detailed explanation of the Tree Search algorithm implementation. Before introducing the code, a small section of imports and helper functions.

## Import setup

In [1]:
import copy
import itertools
import typing
import random

import Greedy_MOD
import Intermediate

from Greedy_MOD import values

from pandas import NA
from pandas._libs.missing import NAType

## Helper Functions and Variables

The `print_attributes` function will help debugging the code: for each programmer-defined class, it prints all its attributes, allowing to monitor the execution of the program.

While it has been extensively used for coding, it has been (almost) completely removed from this version of the notebook.

In [2]:
def print_attributes(obj):
    '''
    Traverses all attributes of an object and prints their name and its corresponding value.
    '''
    for attr in vars(obj):
        print(attr, getattr(obj, attr))

### Test variables

This section contains a list of variables and game scenarios which will be used to travel along all the classes and methods to explain how we engineered the algorithm, allowing the computer to know what a Scopone game is, what is a legal move, what is a card, and, most importantly, how to learn and execute the move that achieves the best score.

In [3]:
PlayerPositions = [0, 1, 2, 3]

#### `test1`

This will be the main game scenario, to which most (if not all) methods will be applied to show how each class and method works and how to structure them properly.

It is a quite complex situation: the AI player starts and can choose between a _single pick_ or different _combinations_.

To summarise:
- 3 turns to the end.
- Multiple combinations available, as well as single picks.
- Montecarlo starts.

In this first definition of a game scenario, we will continue by using the `tuple` to represent a _card_ and the _list of tuple_ for a _set of cards_.

We need to ask ourself a question: how is a _scopone_ game defined? We can model it as a _succession of __states___ and every state is perfectly known if the following informations are available:

1. Player position.
2. Cards on the table.
3. Cards in each players' hand.
4. Each team scores.

By combining these informations we have all we need to know to give each player all he/it needs to decide what card to play.

In [4]:
test1_Table = [
    (1, 1),
    (4, 2),
    (9, 4),
    (6, 3)
]
test1_PlayersCards = {
    0: [(1, 2), (10, 4), (5, 3)],
    1: [(2, 2), (9, 3), (8, 3)],
    2: [(1, 3), (9, 2), (4, 3)],
    3: [(10, 1), (10, 3), (3, 3)]
}
test1_Team = "Hand"
test1_TeamScores= {
    "Hand": 500,
    "Deck": 500
}

### `test2`

This test scenario is for _debug_ purposes: if there is a _scopa_ on the table, it is always the best choice to make it.

We wanted to be sure that the AI was able to recognise this opportunity and to compute the resulting score correctly.

To summarise:

- Same as `test1`, but with a possible _scopa_ to be made

### `test3`

This is a completely different game situation for our AI player: it is the _last_ to play and there are _no cards_ on the table.

This would be difficult even for a competent human player, because the risk of exposing his team to a _scopa_ is very high: therefore, it is a very ambiguous situation.

To summarise:

- Last turn, _Deck_ team.
- Montecarlo is the last one to play.
- No cards on the table.

In [5]:
test3_Table = [
]
test3_PlayersCards = {
    0: [(1, 2), (10, 4), (5, 3)],
    1: [(2, 2), (9, 3), (8, 3)],
    2: [(1, 1), (9, 2), (4, 3)],
    3: [(10, 1), (10, 3), (3, 3), (7, 3)]
}
test3_PlayerPosition = 3
test3_Team = "Deck"
test3_TeamScores= {
    "Hand": 500,
    "Deck": 500
}

### `test4`, `test5`, `test6`

All these test scenarios have been used only for _debugging_ purposes: they simulate a specific scenario in the _game tree_ in which the code was not working properly, allowing to analyse it in a detailed way.

In [6]:
test4_Table = [
    (5, 3),
    (6, 4),
    (9, 1)
]

test4_PlayersCards = {
    0: [(1, 2)],
    1: [(7, 2)],
    2: [(0, 1)],
    3: [(10, 1), (2, 3)]
}
test4_PlayerPosition = 3
test4_Team = "Deck"
test4_TeamScores= {
    "Hand": 500,
    "Deck": 500
}

In [7]:
test5_Table = [
    (10, 3),
    (1, 2)
]

test5_PlayersCards = {
    0: [(1, 2)],
    1: [(2, 2), (8, 3)],
    2: [(9, 2), (4, 3)],
    3: [(10, 1), (3, 3)]
}
test5_PlayerPosition = 0
test5_Team = "Hand"
test5_TeamScores= {
    "Hand": 1584,
    "Deck": 520
}

In [8]:
test6_Table = [
    (6, 3),
    (10, 3)
]

test6_PlayersCards =  {
    0: [(5, 3)],
    1: [(2, 2), (8, 3)],
    2: [(1, 3), (9, 2)],
    3: [(10, 1), (3, 3)]
}

test6_PlayerPosition = 0
test6_Team = "Hand"
test6_TeamScores= {
    "Hand": 570,
    "Deck": 520
}

## Interfaces

> Interfaces are abstract generic classes that contains the required blocks to implement a Monte Carlo Tree Search-based AI player.

Building on Di Palma, S. "_Monte Carlo Tree Search algorithms applied to the card game Scopone_" (2014), we coded this interfaces to have a fundamental _architecture_ that allowed us to keep track of all the blocks needed for the algorithm to work.

From a quick glance, it is clear that the Monte Carlo Tree Search is a whole different class of AI, compared to the `Greedy` and `Intermediate` ruled-based AIs, because exploring the game tree requires: on the one hand, to know everything about the current scenario (_cards, hands, table, score, etc._), and, on the other hand, to _simulate_ a reasonable strategy for every other player, on the same team and on the other.

This implies that we need two fundamental classes: a class that represents a _game state_, `IGameState`, and a class to represent what is the other player's strategies and to simulate their moves, `IGameMove`.

Moreover, we made an architectural choice: each node in the tree will be represented by an instance of the class `IGameState`: therefore, this class will also contain methods and utilities necessary to perform the tree search (e.g.: since each node in the tree has to be _hashable_, the `IGameState` class has a `__hash__` method to achieve this).

In [9]:
class IGameState(object):
    '''
    The GameState wholly defines the game at a given moment.

    Moreover, passes informations from the current game state to the following one.
    '''

    def __init__(self) -> None:
        pass

    def __hash__(self) -> int:
        '''
        GameStates also play the role of nodes in the tree building. Nodes should be hashable.
        '''
        return 19

    def GetAvailableMoves(
        self
    ):
        raise NotImplementedError()

    def ApplyMove(
        self
    ):
        raise NotImplementedError()

    def IsTerminal(
        self
    ):
        '''
        Check if a terminal node is reached (10 turns have passed and this is the last turn).
        '''
        raise NotImplementedError()

    def GetScores(
        self
    ):
        raise NotImplementedError()

    def LastPlayer(
        self
    ):
        '''
        This methods know the last player to move.
        '''
        raise NotImplementedError()

    def CloneState(
        self
    ):
        '''
        This method clones the current IGameState.
        '''
        raise NotImplementedError()


In [10]:
class IGameMove(object):
    '''
    This interface class represents a MOVE in the game.

    It should contain the definition of legal moves
    and routines to evaluate them.

    It should be able to be passed as an argument to the IGameState and update its general state.

    It should be the output of the ITreeNode object that performs a tree search among the possible moves and chooses the optimal one.
    '''

    def __init__(self) -> None:
        pass

    def GetMoves(self) -> None:
        '''Simulate another player's strategy and returns a move.'''
        raise NotImplementedError()

    def EvalMoves(
        self
    ):
        '''Checks if two moves are equal.'''
        raise NotImplementedError()

A final note: these are __generic__ classes. The actual implementation for a _Scopone_ game will be built by exploiting _inheritance_, allowing us to keep track of all the requirements but to instantiate and code all we need to build a working _Scopone_-playing AI.

# Scopone classes

As mentioned earlier, the Monte Carlo Tree Search algorithm (from now on, _MCTS_) needs to "understand" everything about the game and the scenario it finds itself in.

The fundamental block of a game is a __card__ and, while using `tuple`s served us well so far, it is not enough to move forward: the first step, therefore, is to code a `Card` class that allows us to have both custom methods to invoke, a compact _container_ in which to store a complete description of the card, and to code a _custom behaviour_ by using __operator overloading__ and other __magic methods__.



## `Card`

In [11]:
class Card(object):
    """
    This object represent a card, which is defined by:

    - Rank: the number corresponding to the card.
    - Suit (Seme): the suit to which it belongs.
    """

    def __init__(
        self,
        Rank: int = NA,
        Suit: int = NA
        ) -> None:
        """
        To instantiate a Card object, a Rank and Suit have to be specified.
        It is possible to instantiate an "empty" card, in which both Rank and Suits NA.

        Args:
            Rank (int, optional): Defaults to NA.
            Suit (int, optional): Defaults to NA.
        """
        self.Rank = Rank
        self.Suit = Suit

    def __str__(self) -> str:
        """
        This method allows to obtain a custom representation of a Card object when passed as an argument to print.
        """
        Suits = {
            1: "Ori",
            2: "Coppe",
            3: "Spade",
            4: "Bastoni"
        }
        return f"{self.Rank} of {Suits[self.Suit]}, valued {self.Value()} rewards."

    def __repr__(self) -> str:
        """
        This method is used to obtain a compact representation of a Card object.
        """
        return f"Card: ({self.Rank}, {self.Suit})"

    def __hash__(self) -> int:
        """
        Card objects need to be hashable.
        """
        return 666

    def __iter__(self) -> list:
        """
        Card objects are iterable, only once, returning theirself.
        """
        self.n = 0
        return self

    def __next__(self) -> None:
        """
        Card objects are iterable, only once, returning theirself.
        """
        if self.n < 1:
            self.n += 1
            return self
        else:
            raise StopIteration

    def __len__(self) -> int:
        """
        Card objects have length 1.
        """
        return 1

    def Rank(self) -> int:
        return self.Rank

    def Suit(self) -> int:
        return self.Suit

    def Value(self) -> int:
        """
        This method is used to store and retrieve the reward associated with a Card.
        """
        values = {(1, 1): 26, (2, 1): 22, (3, 1): 23, (4, 1): 24, (5, 1): 25, (6, 1): 28, (7, 1): 139, (8, 1): 20, (9, 1): 20, (10, 1): 139,
          (1, 2): 16, (2, 2): 12, (3, 2): 13, (4, 2): 14, (5, 2): 15, (6, 2): 18, (7, 2): 29, (8, 2): 10, (9, 2): 10, (10, 2): 10,
          (1, 3): 16, (2, 3): 12, (3, 3): 13, (4, 3): 14, (5, 3): 15, (6, 3): 18, (7, 3): 29, (8, 3): 10, (9, 3): 10, (10, 3): 10,
          (1, 4): 16, (2, 4): 12, (3, 4): 13, (4, 4): 14, (5, 4): 15, (6, 4): 18, (7, 4): 29, (8, 4): 10, (9, 4): 10, (10, 4): 10}
        return values[self.Rank, self.Suit]

    def __add__(self, other) -> int:
        if isinstance(other, Card):
            return self.Rank + other.Rank
        raise TypeError(
            (f"unsupported operand type(s) for +: "
             f"{type(self)} and {type(other)}"))

    def __radd__(self, other) -> int:
        if other == 0:
            return self
        return self.__add__(other)

    def __mul__(self, other) -> int:
        if isinstance(other, Card):
            return self.Value() + other.Value()
        raise TypeError(
            (f"unsupported operand type(s) for *: "
             f"{type(self)} and {type(other)}"))

    def __eq__(self, other) -> bool:
        if isinstance(self, Card) and isinstance(other, Card):
            if self.Rank == other.Rank and self.Suit == other.Suit:
                return True
            else:
                return False
        elif isinstance(self, tuple) and isinstance(other, Card):
            rank, suit = other.Rank, other.Suit
            if self[0] == rank and self[1] == suit:
                return True
            else:
                return False
        elif isinstance(self, Card) and isinstance(other, tuple):
            rank, suit = self.Rank, self.Suit
            if other[0] == rank and other[1] == suit:
                return True
            else:
                return False


Now, a brief description of how a `Card` works, with some examples.

---

A `Card` object can be instantiated without assigning any attribute:

In [12]:
my_card = Card()
print_attributes(my_card)

Rank <NA>
Suit <NA>


`print` works in a specific way when associated with a `Card`:

In [13]:

my_card.Rank, my_card.Suit = (7, 2)

print(my_card)


7 of Coppe, valued 29 rewards.


To extract the _game-point_ (__reward__) value for a card, invoke the `Value()` method:

In [14]:
my_card.Value()

29

To sum the __ranks__ of two cards, you can use `+`:

In [15]:
card1 = Card(2, 1)
card2 = Card(2, 3)

card1 + card2

4

To sum their __rewards__ values, use `*`.

E.g.: (2, 1) and (2, 3) are valued both 12 _rewards_:

In [16]:
card1*card2

34

To check whether two cards are equal, you can use the __boolean__ operator `==`:

In [17]:
card3 = Card(2, 1)

card1 == card3

True

This works also if we want to compare the `Card` representation with a `tuple` representation of the same card; this is required to allow the MCTS to work with `Greedy` or `Intermediate` based players.

In [18]:
Card(2,1) == (2,1)

True

Note that a `Card` object can be _iterated_, but only once:

In [19]:
for iterables in card1:
    print(iterables)

2 of Ori, valued 22 rewards.


To conclude, we coded other _helper functions_ based on the `Card` objects that allow us to translate between _tuples_ and `Card`s effortlessly.

To convert a `tuple` into a `Card` object:

In [20]:
def convert_to_card(
    card: tuple
) -> Card:
    """
    This routine convert tuple object into Card object.
    If the object is already a Card, it returns its argument unmodified.

    Args:
        card (tuple): a tuple describing a card, (rank, suit).

    Returns:
        Card: a Card describing a card.
    """

    if not isinstance(card, Card):

        new_card = Card()
        new_card.Rank, new_card.Suit = card

        return new_card

    elif isinstance(card, Card):
        return card

In [21]:
print(convert_to_card((2, 1)))

2 of Ori, valued 22 rewards.


To convert a `Card` into a `tuple`:

In [22]:
def convert_to_tuple(
    card: Card
    ) -> tuple:

    temp = card.Rank, card.Suit
    return temp

Other useful functions that require the definition of `Card` objects to work involve:

- A routine to sum `Card`s `Rank`s in some specific scenarios.
- A routine to `unpack_moves`.

In [23]:
def sum_combination_ranks(
    CardCombination: list
    ):
    """
    Given a combination of cards, returns the sum of their ranks.
    To be used to compare the Rank of a move (Card from a player's Hand) and the total Rank represented by the combination,
    to understand possible picks and compare.

    Args:
        CardCombination (list): a list of Card objects.

    Returns:
        int: sum of each Card object rank.
    """

    combination_sum = sum(card.Rank for card in CardCombination)

    return combination_sum


In [24]:
def unpack_moves(MovesDict: dict) -> dict:
    """
    This unpack a moves dictionary by yielding a simpler data structure.
    Args:
        MovesDict (dict): a dictionary of moves

            key: card to be played.
            values: card or combination of cards to be taken from the table.

    Yields:
        dict: a sequence of dictionaries each representing a single move.
    """

    for key in MovesDict.keys():
        if len(MovesDict[key]) == 0:
                yield {key: list()}
        else:
            for move in MovesDict[key]:
                if isinstance(move, Card):
                    yield {key: move}

                elif isinstance(move, list):
                    if key.Rank == sum_combination_ranks(move) or len(move) == 0:
                        yield {key: move}

To represent and __update__ a game state, it is not enough to return a `move`: while simulating the game, the MCTS needs also to modify the `Table`, by adding discarded cards or removing picks. This `unpack_moves` method uses `yield` to create an __iterator__ object that contains a sequence of _dictionaries_. Each of these dictionaries is structured in this way:

- `key`: the `Card` to be played.
- `values`: the `Card` or a list representing a combination of cards to be picked. An empty list represent a situation in which no picks can be made, therefore the player has to discard a card from his hand.

## `ScoponeMove`

`ScoponeMove` inherits from `IGameMove` and its main duty is to simulate the other players' choices, while also containing _utilities_ to evaluate moves and get possible combinations.

To simulate the other players' strategy, we needed a model of their behaviour. To do so, this class effectively works as a _wrapper_ for the `Greedy` and `Intermediate` functions, which are chosen to model the response to the MCTS player possible choices.

### Code

In [25]:
class ScoponeMove(IGameMove):
    '''
    This class inherits from IGameMove and, given the rules of Scopone and the definition of the current IGameState,
    either apply a strategy to select which card to play or evaluates whether two cards yield the same points.
    '''

    def __init__(
        self,
        LegalMoves: list = [],
        Table: list = [],
        Deck: list = [],
        values: dict = Greedy_MOD.values,
        ) -> None:
        """
        This section assigns all arguments needed to define the current state of the game.

        Args:
            values (dict, optional): a dictionary that contains the points associated with each card. Defaults to Greedy_MOD.values.
            LegalMoves (list, optional): A list of tuples containing the player's hand. Defaults to [].
            Table (list, optional): A list of tuples containing all the cards on the table. Defaults to [].
            Deck (list, optional): A list of tuples containing all the cards on the deck. Defaults to [].
        """

        super().__init__()

        self.Hand = LegalMoves
        self.Table = Table
        self.Deck = Deck
        self.values = values

    def GetMove(
        self,
        Greedy: bool = True,
        Standalone:bool = False
        ) -> tuple:
        """
        This method, given the elements of the game (LegalMoves, Table, Deck), outputs a tuple representing the card chosen by applying the Intermediate or Greedy (default) strategies.

        Args:
            game_mode (bool, optional): Decide whether to use the Greedy strategy or not. Defaults to False.

        Returns:
            tuple: a card, representing the move chosen by the AI.

        Yields:
            Iterator[tuple]: a card, as defined by points and rank.
        """

        if Greedy:
            return Greedy_MOD.Greedy(
                legalMoves = self.Hand,
                table = self.Table,
                Standalone=Standalone,
                values = values
                )

        else:
            return Intermediate.Intermediate(
                legalMoves = self.Hand,
                table = self.Table,
                deck = self.Deck,
                Standalone=Standalone,
                values = values
                )

    def GetCombinations(
        Table: list
        ) -> dict:
        """
        This method returns all possible combinations of table cards

        Args:
            Table (tuple): a list of tuples representing cards.

        Returns:
            Combinations: a dictionary of summed ranks as keys and combinations of cards as values representing the legal combined picks.
        """

        available_combinations = []

        for L in range(2, len(Table) + 1):
            available_combinations.append(itertools.combinations(map(convert_to_card, Table), L))

        Combinations = {key: list() for key in range(1, 11)}

        for combinations in available_combinations:
            for combination in combinations:
                combination_sum = 0
                combination_cards = []
                for card in combination:
                    combination_sum += card.Rank
                    combination_cards.append(card)
                if combination_sum < 11:
                    Combinations[combination_sum].append(combination_cards)


        return Combinations

    def EvalMoves(
        move1: dict,
        move2: dict
        ) -> tuple:
        """
        This method takes two moves as arguments, in the form:

        move = {card to be played: card or list of cards (combination) to be picked from the table}

        Returns a tuple (bool, dict) with:
            - bool: truth value resulting from the comparison of the total reward corresponding to that move.
            - dict: the best move among the two inputs.

        Args:
            move1 (dict): a dict object representing a move.
            move2 (dict): a dict object representing a move.

        Returns:
            tuple(bool, dict):
            - bool: truth value resulting from the comparison of the total reward corresponding to that move.
            - dict: the best move among the two inputs.
        """

        move1_cards_list = list()
        move1_tot_reward = 0

        for move, combinations in move1.items():
            move1_cards_list.append(convert_to_card(move))
            if isinstance(combinations, list):
                for pick in combinations:
                    move1_cards_list.append(convert_to_card(pick))
            if isinstance(combinations, tuple):
                move1_cards_list.append(convert_to_card(combinations))


        move1_tot_reward = sum(card.Value() for card in move1_cards_list)

        move2_cards_list = list()
        move2_tot_reward = 0

        for move, combinations in move2.items():
            move2_cards_list.append(convert_to_card(move))
            if isinstance(combinations, list):
                for pick in combinations:
                    move2_cards_list.append(convert_to_card(pick))
            if isinstance(combinations, tuple):
                move2_cards_list.append(convert_to_card(combinations))

        move2_tot_reward = sum(card.Value() for card in move2_cards_list)

        if move1_tot_reward > move2_tot_reward:
            BestMove = move1
        elif move1_tot_reward < move2_tot_reward:
            BestMove = move2
        else:
            # If two cards yield the same value, flip a coin to decide which one to play.
            diceroll = random.randint(0, 1000)
            if diceroll < 500:
                BestMove = move1
            else:
                BestMove = move2

        return move1_tot_reward == move2_tot_reward, BestMove

---
### Test

#### `GetMove`


##### `Intermediate`:

In [26]:
ScoponeMove(
    LegalMoves=test1_PlayersCards[0],
    Table= test1_Table
    ).GetMove(
        Greedy=False,
        Standalone=False
)

{(5, 3): [(1, 1), (4, 2)]}

In [27]:
%timeit ScoponeMove(LegalMoves=test1_PlayersCards[0], Table=test1_Table ).GetMove(Greedy=False, Standalone=False)

34.7 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


##### `Greedy`:

The following code cell tests the `Greedy` inside the `GetMove` wrapper method.

In [28]:
ScoponeMove(
    LegalMoves=test1_PlayersCards[0],
    Table= test1_Table
    ).GetMove(
        Greedy=True,
        Standalone=False
)

{(5, 3): [(1, 1), (4, 2)]}

In [29]:
%timeit ScoponeMove(LegalMoves=test1_PlayersCards[0], Table=test1_Table ).GetMove(Greedy=True, Standalone=False)

16.8 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


> __Spoiler__: this is _not_ the best move, given the whole game tree.

As we will see later on, the MCTS consistently plays a different strategy than either the `Intermediate` and `Greedy` AIs, in this case, taking the 6 and 4 in the table with the 10 in its hand: even though it rewards less point immediatly, it will lead to worse outcomes by the end of the game.

#### `GetCombinations`

When facing a complex game scenario, such as `test_1`, in which with a single card you can pick alternative singles or combinations from the table.

To test the `GetCombinations` method, we modified the table by adding a single picks. In other words, if the player has a (10, 3), this method consider all the alternatives and outputs a dictionary in which the key represent a `Rank` and each value is updated if a corresponding combination add up to the played card `Rank`.

In [30]:
Table=[(1, 1), (4, 2), (9, 4), (10, 2), (6, 3)]

In [31]:
ScoponeMove.GetCombinations(Table)

{1: [],
 2: [],
 3: [],
 4: [],
 5: [[Card: (1, 1), Card: (4, 2)]],
 6: [],
 7: [[Card: (1, 1), Card: (6, 3)]],
 8: [],
 9: [],
 10: [[Card: (1, 1), Card: (9, 4)], [Card: (4, 2), Card: (6, 3)]]}

#### `EvalMoves`

In [32]:
move1 = {
    (10, 1): [
        (6, 3),
        (3, 4)
        ]
    }
move2 = {
    (10, 2):
        (10, 1)
    }
move3 = {
    (10, 1): [
        (6, 2),
        (3, 4)
        ]
    }
move4 = {
    (10, 1): [
        (7, 1),
        (3, 1)
        ]
    }

`move1` and `move3` yield equal rewards; the worst move is `move2`, the best `move4`.

> Note that this is not an actual game scenario made of legal moves, but just test.

In [33]:

ScoponeMove.EvalMoves(
    move1,
    move3
)

(True, {(10, 1): [(6, 3), (3, 4)]})

In [34]:
ScoponeMove.EvalMoves(
    move1,
    move2
)

(False, {(10, 1): [(6, 3), (3, 4)]})

In [35]:
ScoponeMove.EvalMoves(
    move1,
    move4
)

(False, {(10, 1): [(7, 1), (3, 1)]})

## `ScoponeGameState`

This class inherits from `IGameState` and develops it extensively. The main reason is that we made a fundamental design choice: the `ScoponeGameState` not only works as a wrapper for everything that the MCTS would need to explore the game tree and simulate other players' strategy, updating the game as it goes on searching for the best move, but it also represents (conceptually) a __node__ in the game tree.

In other words, we decided that since a __game state__ can be thought and modelled as a specific point in the game, it also can work as the coded representation of a __node__.

This choice makes for a complex object, full of different methods and with a lengthy list of attributes: everything it needs to define a scenario, apply the best move (modifying both its hand and the table and updating the scores), check if the game is in its last turn, find the children of a node (game state), store the information about the node's parents, and so on. Moreover, we made `ScoponeGameStates` _hashable_, as it is a requirement for tree nodes.

### Code

In [36]:
class ScoponeGameState(IGameState):
    """
    This object completely defines the current game 'state' situation.

    It contains methods to:

    - get the available moves,
    - apply a move,
    - check if it is a terminal state,
    - get the scores fo the game,
    - know the last player to move,
    - clone the state.
    - compute the rewards.
    - find all the children of the current ScoponeGameState.
    - find a random children of the current ScoponeGameState.
    """
    # def __new__(cls, *args, **kwargs):
    #     print("####### I have created a new ScoponeGameState.")
    #     return super().__new__(cls)

    def __init__(
        self,
        PlayerPosition: int,
        PlayersCards: dict,
        Team: str,
        TeamScores: dict,
        Table: list = [],
        Deck: list = [],
        LastTaker: int = NA,
        ParentMove: dict = NA,
        values: dict = Greedy_MOD.values
        ) -> None:
        """
        This section assigns all arguments needed to define the current state of the game, for all players.

        Args:
            PlayerPosition (int): the position of the current player. Might be a number from 0 to 3, based on the starting turn.
            PlayersCards (dict): a dictionary containing a list of tuples representing each player's hand.
            Team(str): a string representing the current player's team.
            TeamScores (dict): a dictionary representing team A (player 0, 2) and team B (player 1, 3) scores.
            Table (list, optional): a list of tuples containing all the cards on the table. Defaults to [].
            Deck (list, optional): a list of tuples containing all the cards on the deck Defaults to [].
            LastTaker(int, optional): an integer indicator that stores the position of the last player that has picked cards from the table.
            ParentMove(dict, optional): a dictionary representing the MCTS move that originated the current GameState.
            values (dict, optional): points associated to each card. Defaults to Greedy_MOD.values.
        """

        self.PlayerPosition = PlayerPosition
        self.PlayersCards = PlayersCards
        self.Team = Team
        self.Hand = PlayersCards[PlayerPosition]
        self.TeamScores = TeamScores
        self.Reward = self.ComputeRewards()
        self.Table = Table
        self.Deck = Deck
        self.LastTaker = LastTaker
        self.ParentMove = ParentMove
        self.values = values

    def __eq__(self, other) -> bool:
        check = [self.PlayerPosition == other.PlayerPosition,
                self.PlayersCards == other.PlayersCards,
                self.Team == other.Team,
                self.Hand == other.Hand,
                self.TeamScores == other.TeamScores,
                self.Table == other.Table,
                self.Deck == other.Deck,
                self.values == other.values
        ]

        if all(check) == True:
            return True
        else:
            return False

    def __hash__(self) -> int:
        return super().__hash__()

    def __repr__(self) -> str:
        if isinstance(self.ParentMove, Card):
            return f"###\nScoponeGameState:\n###\n>Parent Move: {self.ParentMove}.\n> Current hand: {self.Hand}\n> Current table: {self.Table}\n> Points: {self.TeamScores}\nThe reward for this State is: {self.Reward}.\n###"
        elif isinstance(self.ParentMove, ScoponeGameState):
            state = self.ParentMove
            return f"###\nScoponeGameState:\n###\n>Parent State: {state.ParentMove}.\n> Current hand: {state.Hand}\n> Current table: {state.Table}\n> Points: {state.TeamScores}\nThe reward for this State is: {state.Reward}.\n###"
        else:
            return f"###\nScoponeGameState:\n###\n>Parent Move: {self.ParentMove}.\n> Current hand: {self.Hand}\n> Current table: {self.Table}\n> Points: {self.TeamScores}\nThe reward for this State is: {self.Reward}.\n###"

    def CloneState(self) -> IGameState:
        """
        This method creates a copy of the current game state and all its attributes.

        Returns:
            ScoponeGameState: a deepcopy of the current ScoponeGameState.
        """

        return copy.deepcopy(self)

    def IsTerminal(self) -> bool:
        """
        This is method to check whether the GameState corresponds to the last turn.

        Returns:
            bool: True if it is the last turn, otherwise False.
        """

        if len(self.Hand) == 0:
            return True
        else:
            return False

    def GetScores(self) -> tuple:
        """
        This EVEN BETTER method gets all the available scores to be rewarded to the player at the current game state.
        Given a table and the player's hand, it will return a list of scores, a dictionary of single picks moves, and a dictionary of combinations.

        Returns:
            A TUPLE with:

            Scores: A list of all the scores available for the taking.
            SinglePicks: A dictionary of all the single-pick moves that can be made and the correspondent points.
                - The dictionary key correspond to the rank, while the value is the card.
            TableCombinations: A list of all the combinations of cards that can be taken and their correspondent point.
                - The dictionary key correspond to the sum of the ranks of all the card in the combinations, while the values are the cards in that particular combination.
        """

        SinglePicks = {card.Rank: card for card in map(convert_to_card, self.Table)}

        TableCombinations = ScoponeMove.GetCombinations(self.Table)

        Scores = []

        for card in SinglePicks.values():
            Scores.append(card.Value())

        for combinations_list in TableCombinations.values():
            total_score = 0
            for combination in combinations_list:
                for card in combination:
                    total_score *= card.Value()
                Scores.append(total_score)

        return Scores, SinglePicks, TableCombinations

    def ComputeRewards(self) -> int:
        """
        This method takes the two teams' scores and computes their difference.

        Points scored for the player's team are positive integers, while the other's team are negative: this
        is used to teach the algorithm the correct behaviour in a reinforcement learning setting.

        Returns:
            int: the difference between the player's and the other's teams scores.
        """

        if self.Team == "Hand":
            StateReward = self.TeamScores["Hand"] - self.TeamScores["Deck"]
        elif self.Team == "Deck":
            StateReward = self.TeamScores["Deck"] - self.TeamScores["Hand"]

        return StateReward


    def GetAvailableMoves(self) -> list[dict]:
        """
        This method indicates all the available cards to be played by the current player.

        Moves are represented by the card played to make that move.
        Given a game state, it will return all the available moves, single picks and combinations.

        Returns:
            list: list of CARDS that can be played at that particular turn.

        IMPORTANT: if a card is present more than once, it might pick either a single card or a combination.
        """


        AvailableMoves = {card: card for card in map(convert_to_card, self.Hand)}


        _, SinglePicks, TableCombinations = self.GetScores()


        LegalMoves = {move: list() for move in AvailableMoves.values()}

        for move in AvailableMoves.values():
            for pick in SinglePicks.values():
                if move.Rank == pick.Rank:
                    LegalMoves[move].append(pick)

        for combinations in TableCombinations.values():
            for combination in combinations:
                for move in LegalMoves.keys():
                    if move.Rank == sum_combination_ranks(combination):
                        LegalMoves[move].append(combination)

        return LegalMoves

    def ApplyMove(
            self,
            BestMove: dict
            ):
        """
        Given a move in the following format:

        {card to be played: card or combination of cards to be taken}

        returns a modified GameState object that applies it to the current GameState.
        This implies:

        - Removing the card from the player's hand.
        - Removing the picked card/s from the table.
        - Adding the resulting reward to that team score.

        Args:
            Move (dict): a dictionary in the form {card to be played: card or combination of cards to be taken}.
            The combination of cards is a list of tuples/Cards objects.

            - Move key = card in your hand to be played. Also called move.
            - Move value = card/combinations of cards to be taken from the table. Also called pick.
                           A pick can either be a combination or a single card. Single cards are iterable once.

        Returns:
            IGameState: a copy of the starting GameState modified by the effects of the BestMove.
        """

        # Reminder:
        #
        # BestMove key = card in your hand to be played.
        # BestMove value = card/combinations of cards to be taken from the table.

        new_game_state = self.CloneState()
        new_table = list(map(convert_to_card, new_game_state.Table))
        new_hand = list(map(convert_to_card, new_game_state.Hand))
        new_score = 0
        updated_scores = new_game_state.TeamScores

        picks_check = len(list(BestMove.values())[0])

        if picks_check == 0 or len(new_game_state.Table) == 0:
            card_to_be_played = [card for card in map(convert_to_card, BestMove.keys())][0]

            new_table.append(card_to_be_played)
            new_hand.remove(card_to_be_played)

            new_game_state.Hand = list(map(convert_to_tuple, new_hand))
            new_game_state.PlayersCards[new_game_state.PlayerPosition] = new_game_state.Hand
            new_game_state.Table = list(map(convert_to_tuple, new_table))

            return new_game_state

        else:
            self.LastTaker = new_game_state.PlayerPosition
            for card_to_be_taken in BestMove.values():
                card_to_be_played = list(map(convert_to_card, [BestMove.keys()][0]))


                for card in new_hand:
                    # scorre tutte le carte in mano e toglie dalla lista la carta che vuole giocare
                    for move in card_to_be_played:
                        if move == card:
                            new_hand.remove(card)
                            new_score += card.Value()



                    for card in card_to_be_taken:
                        #scorre tutte le carte in tavola e toglie dalla lista tavolo la carta/combinazione di carte
                        if isinstance(card_to_be_taken, tuple):
                            card_to_be_taken = convert_to_card(card_to_be_taken)
                        if isinstance(card, int):
                            card = card_to_be_taken
                        for pick in new_table:
                            if isinstance(pick, list):
                                for card_to_remove in pick:
                                    if card_to_remove == card:
                                        new_table.remove(pick)
                                        new_score += pick.Value()
                            elif isinstance(pick, Card) and pick == card:
                                new_table.remove(pick)
                                new_score += pick.Value()
                            elif isinstance(pick, tuple) and pick == card:
                                pick = convert_to_card(pick)
                                new_table.remove(pick)
                                new_score += pick.Value()

            if len(new_table) == 0 and not self.IsTerminal():
                #aggiungi un punto perché hai fatto scopa
                new_score += 1000

            new_players_cards = new_game_state.PlayersCards
            new_players_cards[new_game_state.PlayerPosition] = list(map(convert_to_tuple, new_hand))

            updated_scores[new_game_state.Team] += new_score

            new_game_state.PlayersCards = new_players_cards
            new_game_state.Hand = new_hand
            new_game_state.TeamScores = updated_scores
            new_game_state.Reward = new_game_state.ComputeRewards()
            new_game_state.Table = list(map(convert_to_tuple, new_table))

            return new_game_state

    def FindChildren(self) -> list:
        """
        This methods applies all available moves and returns a list containing
        each of the corresponding modified GameStates.

        IMPORTANT: this method is invoked to start a tree search as it considers all the possible
        moves.

        Returns:
            list: a list of modified GameStates.
        """

        AvailableMoves = list(unpack_moves(self.GetAvailableMoves()))

        Childrens = list()

        for move in AvailableMoves:
            new_child = self.ApplyMove(move)
            new_child.ParentMove = move
            Childrens.append(new_child)

        return Childrens

    def FindRandomChildren(self) -> IGameState:
        """
        To efficiently simulate a game, this method returns a random ScoponeGameState
        chosen from the available moves.

        Returns:
            IGameState: a ScoponeGameState.
        """

        AllChildren = self.FindChildren()

        RandomChild = AllChildren[random.randint(0, len(AllChildren) -1)]

        return RandomChild


---
### Test

To start our tests, we create our first `ScoponeGameState` representing the game defined as `test_1`:

In [37]:
test1_SGS = ScoponeGameState(
    PlayerPosition=0,
    PlayersCards=test1_PlayersCards,
    Team=test1_Team,
    TeamScores=test1_TeamScores,
    Table=test1_Table
)

Note that creating a new game state prints a warning: this is used for debug purposes and to keep track of the algorithms computations.

To analyse a `ScoponeGameState`, you can either use `print_attributes`, which allows you to have a comprehensive view:

In [38]:
print_attributes(test1_SGS)

PlayerPosition 0
PlayersCards {0: [(1, 2), (10, 4), (5, 3)], 1: [(2, 2), (9, 3), (8, 3)], 2: [(1, 3), (9, 2), (4, 3)], 3: [(10, 1), (10, 3), (3, 3)]}
Team Hand
Hand [(1, 2), (10, 4), (5, 3)]
TeamScores {'Hand': 500, 'Deck': 500}
Reward 0
Table [(1, 1), (4, 2), (9, 4), (6, 3)]
Deck []
LastTaker <NA>
ParentMove <NA>
values {(1, 1): 26, (2, 1): 22, (3, 1): 23, (4, 1): 24, (5, 1): 25, (6, 1): 28, (7, 1): 139, (8, 1): 20, (9, 1): 20, (10, 1): 139, (1, 2): 16, (2, 2): 12, (3, 2): 13, (4, 2): 14, (5, 2): 15, (6, 2): 18, (7, 2): 29, (8, 2): 10, (9, 2): 10, (10, 2): 10, (1, 3): 16, (2, 3): 12, (3, 3): 13, (4, 3): 14, (5, 3): 15, (6, 3): 18, (7, 3): 29, (8, 3): 10, (9, 3): 10, (10, 3): 10, (1, 4): 16, (2, 4): 12, (3, 4): 13, (4, 4): 14, (5, 4): 15, (6, 4): 18, (7, 4): 29, (8, 4): 10, (9, 4): 10, (10, 4): 10}


However, this might result as _verbose_ in most situations: to obtain a more compact representation, call `print` on the `ScoponeGameState` object.

In [39]:
print(test1_SGS)

###
ScoponeGameState:
###
>Parent Move: <NA>.
> Current hand: [(1, 2), (10, 4), (5, 3)]
> Current table: [(1, 1), (4, 2), (9, 4), (6, 3)]
> Points: {'Hand': 500, 'Deck': 500}
The reward for this State is: 0.
###


This yields all the necessary information to evaluate the current situation.

#### `CloneState`

This method is necessary to work in a "_safe_" way: while the computer makes all the computations necessary to evaluate or modify a `ScoponeGameState`, all the modifications need to be implemented on a _new_ instance, to avoid inadvertently modifying the original game state, which is used to perform a different _simulation_ or _rollout_.

Once a `ScoponeGameState` has been created, invoking the methods create a separate copy that can be modified safely.

In [40]:
copy_gamestate_test = test1_SGS.CloneState()

In [41]:
print_attributes(copy_gamestate_test)

PlayerPosition 0
PlayersCards {0: [(1, 2), (10, 4), (5, 3)], 1: [(2, 2), (9, 3), (8, 3)], 2: [(1, 3), (9, 2), (4, 3)], 3: [(10, 1), (10, 3), (3, 3)]}
Team Hand
Hand [(1, 2), (10, 4), (5, 3)]
TeamScores {'Hand': 500, 'Deck': 500}
Reward 0
Table [(1, 1), (4, 2), (9, 4), (6, 3)]
Deck []
LastTaker <NA>
ParentMove <NA>
values {(1, 1): 26, (2, 1): 22, (3, 1): 23, (4, 1): 24, (5, 1): 25, (6, 1): 28, (7, 1): 139, (8, 1): 20, (9, 1): 20, (10, 1): 139, (1, 2): 16, (2, 2): 12, (3, 2): 13, (4, 2): 14, (5, 2): 15, (6, 2): 18, (7, 2): 29, (8, 2): 10, (9, 2): 10, (10, 2): 10, (1, 3): 16, (2, 3): 12, (3, 3): 13, (4, 3): 14, (5, 3): 15, (6, 3): 18, (7, 3): 29, (8, 3): 10, (9, 3): 10, (10, 3): 10, (1, 4): 16, (2, 4): 12, (3, 4): 13, (4, 4): 14, (5, 4): 15, (6, 4): 18, (7, 4): 29, (8, 4): 10, (9, 4): 10, (10, 4): 10}


#### `==` (comparison operator)

This operator overloading, coded by the `__eq__` magic method, allows to use `==` to check whether two `ScoponeGameState` are equal.

In [42]:
copy_gamestate_test == test1_SGS

True

In [43]:
dummy_test1_SGS = ScoponeGameState(
    PlayerPosition=0,
    PlayersCards=test1_PlayersCards,
    Team=test1_Team,
    TeamScores=test1_TeamScores,
    Table=test1_Table
)

dummy_test1_SGS == test1_SGS

True

This will be used extensively while exploring the tree: a __branch_ can be thought as a sequence of _nodes_ (game states); for two paths to be equal, each of these states must have the same parent and lead to the same children (in other words, all the `ScoponeGameState`s that make up the sequence will be equal).

To avoid wasting computational resources, this method is used to check whether a path has been already explored fully and can therefore be skipped.

#### Are `ScoponeGameState`s hashable?

Nodes in the game tree has to be hashable: the following code block checks whether the `__hash__` method works as expected.

In [44]:
isinstance(test1_SGS, typing.Hashable)

True

#### `GetAvailableMoves`

This method is used to create a list of all the available _legal_ moves for the AI player. It is implemented by putting together several already coded methods and utility functions, such as `GetScores` and `convert_to_card`.

As we have seen earlier, the MCTS needs to have a computational _description_ of a Scopone game; however, not necessarily it is interfaced with a human player of another MCTS agent: this means that this method (as `ApplyMove`) needs to be able to operate either with `Card`s, `tuple`s, or both.

Metaphorically, it is the first method in which we see that the class implementation of the algorithm is used to create a way for the program to _understand_ the game to analyse it and make a move by creating a custom _environment_, which at the same time is able to interface with a game setting and rule-based AIs in which the implementation is simpler and do not need a fully constructed information about the game and its rule to play.

First of all, the algorithm computes __all the possibilities__: single picks, combinations, and their relative scores, operating on an instantiated node:

In [45]:
test1_SGS.GetScores()

([26, 14, 10, 18, 0, 0, 0, 0],
 {1: Card: (1, 1), 4: Card: (4, 2), 9: Card: (9, 4), 6: Card: (6, 3)},
 {1: [],
  2: [],
  3: [],
  4: [],
  5: [[Card: (1, 1), Card: (4, 2)]],
  6: [],
  7: [[Card: (1, 1), Card: (6, 3)]],
  8: [],
  9: [],
  10: [[Card: (1, 1), Card: (9, 4)], [Card: (4, 2), Card: (6, 3)]]})

This method outputs a `tuple`, the first element of which is just a computation of the rewards associated with the possible picks or cards to be discarded (discarding a card yields 0 points).

The second item is a dictionary of single picks: each of them has the `Rank` as key and the relative `Card` as value.

The third item is a dictionary of combinations: all possible `Rank` sums are considered and their value correspond to a list of the lists of cards which summed rank represent the legal combination.

Note that all these operations are made by the method by taking into account only the `Table` attribute; the method `ApplyMove` will build on this and bridge the output of this method to create __moves__ with the player's `Hand`.

In other words, this method correctly identifies that, looking at the current game scenario, a player could pick a 1, a 4, a 6, a 9, a 5 (1 + 4) and a 10 (by either combining 1 + 9 or 4 + 6).

`GetAvailableMoves` is the method that builds a bridge between `GetScores` and `ApplyMove`: a __move__ is defined as a `dict` in which the __key__ represent the card to be played or discarded and the value either the single `Card` to be picked (and removed from the `Table`) or the `list` (combination) of cards.

In [46]:
test1_AvailableMoves = test1_SGS.GetAvailableMoves()

In [47]:
test1_AvailableMoves

{Card: (1, 2): [Card: (1, 1)],
 Card: (10, 4): [[Card: (1, 1), Card: (9, 4)], [Card: (4, 2), Card: (6, 3)]],
 Card: (5, 3): [[Card: (1, 1), Card: (4, 2)]]}

The utility function `unpack_moves` comes into play to iterate among this problematic scenario: what if I can choose between different combinations to be picked with the same card?

`dict` keys must be unique, and this makes for a collection of moves which can be hard to iterate upon: as we see in the example, we have a list, a list of lists, and we could also have just a `Card` as the value.

In [48]:
unpack_moves(test1_SGS.GetAvailableMoves())

<generator object unpack_moves at 0x11f8d0890>

This function solve this problem by creating a `generator` that, when iterated, yield just a simple dictionary with a `Card` as key and a `Card` or list of `Card`s as value.

In [49]:
list(unpack_moves(test1_AvailableMoves))

[{Card: (1, 2): Card: (1, 1)},
 {Card: (10, 4): [Card: (1, 1), Card: (9, 4)]},
 {Card: (10, 4): [Card: (4, 2), Card: (6, 3)]},
 {Card: (5, 3): [Card: (1, 1), Card: (4, 2)]}]

If no cards can be taken from the table, these methods works in the same way. Not taking anything is translated as an empty list, which will be the `value` of a dictionary having as `key` the card that the player can discard from its hand.

In [50]:
test3_SGS = ScoponeGameState(
    PlayerPosition=test3_PlayerPosition,
    PlayersCards=test3_PlayersCards,
    Team=test3_Team,
    TeamScores=test3_TeamScores,
    Table=test3_Table
)
print_attributes(test3_SGS)

PlayerPosition 3
PlayersCards {0: [(1, 2), (10, 4), (5, 3)], 1: [(2, 2), (9, 3), (8, 3)], 2: [(1, 1), (9, 2), (4, 3)], 3: [(10, 1), (10, 3), (3, 3), (7, 3)]}
Team Deck
Hand [(10, 1), (10, 3), (3, 3), (7, 3)]
TeamScores {'Hand': 500, 'Deck': 500}
Reward 0
Table []
Deck []
LastTaker <NA>
ParentMove <NA>
values {(1, 1): 26, (2, 1): 22, (3, 1): 23, (4, 1): 24, (5, 1): 25, (6, 1): 28, (7, 1): 139, (8, 1): 20, (9, 1): 20, (10, 1): 139, (1, 2): 16, (2, 2): 12, (3, 2): 13, (4, 2): 14, (5, 2): 15, (6, 2): 18, (7, 2): 29, (8, 2): 10, (9, 2): 10, (10, 2): 10, (1, 3): 16, (2, 3): 12, (3, 3): 13, (4, 3): 14, (5, 3): 15, (6, 3): 18, (7, 3): 29, (8, 3): 10, (9, 3): 10, (10, 3): 10, (1, 4): 16, (2, 4): 12, (3, 4): 13, (4, 4): 14, (5, 4): 15, (6, 4): 18, (7, 4): 29, (8, 4): 10, (9, 4): 10, (10, 4): 10}


In [51]:
test3_SGS.GetAvailableMoves()

{Card: (10, 1): [], Card: (10, 3): [], Card: (3, 3): [], Card: (7, 3): []}

#### `ApplyMove`

`ApplyMove` is the __workhorse__ method in the creation of a simulated game: taking as input a move, chosen systematically or at random, it __applies__ it by modifying:

- the player's `Hand`: the played card is removed.
- the `Table` for all players: if there is a pick to be made, the corresponding single card or set is removed.
- the `Team` score: the team is awarded a reward, which will be either
    - Positive: if the points are taken by the player or its team mate.
    - Negative: if the points are taken by either one of the other players.

A quick reminder of the current game state, `test_1`:

In [52]:
print(test1_SGS)

###
ScoponeGameState:
###
>Parent Move: <NA>.
> Current hand: [(1, 2), (10, 4), (5, 3)]
> Current table: [(1, 1), (4, 2), (9, 4), (6, 3)]
> Points: {'Hand': 500, 'Deck': 500}
The reward for this State is: 0.
###


As an example, we imagine that the AI player wants to take the __9 of Bastoni__ (9, 4) and the __1 or Ori__ (1, 1) with the __10 of Bastoni__:

In [53]:
test1_BestMove = {Card(10, 4): [Card(9, 4), Card(1, 1)]}

test1_modified = test1_SGS.ApplyMove(test1_BestMove)

In [54]:
print(test1_modified)

###
ScoponeGameState:
###
>Parent Move: <NA>.
> Current hand: [Card: (1, 2), Card: (5, 3)]
> Current table: [(4, 2), (6, 3)]
> Points: {'Hand': 546, 'Deck': 500}
The reward for this State is: 46.
###


As we can see, the method __modified__ the `ScoponeGameState`, leading to another __node__ in the game tree.

Both the _hand_ and _table_ have been updated by removing all the corresponding cards; the scores have been awarded to the team to which the player belongs.

If no pick is available:

In [55]:
test3_SGS.ApplyMove(
        {(7, 3): []}
    )

###
ScoponeGameState:
###
>Parent Move: <NA>.
> Current hand: [(10, 1), (10, 3), (3, 3)]
> Current table: [(7, 3)]
> Points: {'Hand': 500, 'Deck': 500}
The reward for this State is: 0.
###

Only the `Hand` attribute is modified. While this is also a _node_, no points are awarded.

Note that the `ParentMove` attribute is not updated: this will be the task assigned to another class, because `ApplyMove` is used with moves originated by different algorithms, both the rule-based simulated players and the MCTS; however, we are only interested in the move made by the MCTS agent for the __backpropagation__ algorithm; therefore, `ParentMove` will be updated elsewhere.

#### `FindChildren`

The Monte Carlo algorithm performs a search among all the branches of a game tree. If a `ScoponeGameState` is a computational representation of a __node__, it will branch out generating a number of `Children`: each of these is a new `ScoponeGameState`, created by computing and simulating the consequences of playing one among the possible moves in that given scenario.

The possible moves to be made in the game state `test1` are:

In [56]:
list(unpack_moves(test1_AvailableMoves))

[{Card: (1, 2): Card: (1, 1)},
 {Card: (10, 4): [Card: (1, 1), Card: (9, 4)]},
 {Card: (10, 4): [Card: (4, 2), Card: (6, 3)]},
 {Card: (5, 3): [Card: (1, 1), Card: (4, 2)]}]

Each one of them can be conceptualised as the `Parent` node of a `Children` node: `FindChilden` is the method that finds them and creates a list of the resulting game states.

In [57]:

test1_childrens = test1_SGS.FindChildren()

In [58]:
test1_childrens

[###
 ScoponeGameState:
 ###
 >Parent Move: {Card: (1, 2): Card: (1, 1)}.
 > Current hand: [Card: (10, 4), Card: (5, 3)]
 > Current table: [(4, 2), (9, 4), (6, 3)]
 > Points: {'Hand': 542, 'Deck': 500}
 The reward for this State is: 42.
 ###,
 ###
 ScoponeGameState:
 ###
 >Parent Move: {Card: (10, 4): [Card: (1, 1), Card: (9, 4)]}.
 > Current hand: [Card: (1, 2), Card: (5, 3)]
 > Current table: [(4, 2), (6, 3)]
 > Points: {'Hand': 546, 'Deck': 500}
 The reward for this State is: 46.
 ###,
 ###
 ScoponeGameState:
 ###
 >Parent Move: {Card: (10, 4): [Card: (4, 2), Card: (6, 3)]}.
 > Current hand: [Card: (1, 2), Card: (5, 3)]
 > Current table: [(1, 1), (9, 4)]
 > Points: {'Hand': 542, 'Deck': 500}
 The reward for this State is: 42.
 ###,
 ###
 ScoponeGameState:
 ###
 >Parent Move: {Card: (5, 3): [Card: (1, 1), Card: (4, 2)]}.
 > Current hand: [Card: (1, 2), Card: (10, 4)]
 > Current table: [(9, 4), (6, 3)]
 > Points: {'Hand': 555, 'Deck': 500}
 The reward for this State is: 55.
 ###]

`FindChildren`, moreover, being the fundamental building block of the MCTS, is only used in that setting: as we can see, now also the `ParentMove` attribute is updated. This allows us to keep track of the sequence (path) composing a branch.

#### `FindRandomChildren`

However, finding __all__ the possible childrens is not enough: the Monte Carlo method is __simulation__ based. To perform a rollout, to fully explore a brach, it needs to __randomly__ select a _children: this allows us to obtain a simplified tree, in which each node does not lead to multiple branches because every time a random choice is made to rollout only a single possible path.

In [59]:
test1_SGS.FindRandomChildren()

###
ScoponeGameState:
###
>Parent Move: {Card: (10, 4): [Card: (1, 1), Card: (9, 4)]}.
> Current hand: [Card: (1, 2), Card: (5, 3)]
> Current table: [(4, 2), (6, 3)]
> Points: {'Hand': 546, 'Deck': 500}
The reward for this State is: 46.
###

## `MCTS`

> __DISCLAIMER__: from this point onward, the mental stability of the coders was severely depleted, causing a 56.783% decrease in the level of seriousness.

The `ScoponeMove` and `ScoponeGameState` classes are the two main ingredients from which an AI player can:

- Store and retrieve all the information needed to computationally represent a game scenario at any given time.
- Modifiy this scenario by exploring the possible ramifications of either its moves and the reasonable opponents' and team members strategies.

The last missing item is a _pot_, another container that implements the Monte Carlo Tree seach by mixing together these ingredients, adding all the necessary tools and routines to make everything work together.

We created a __MCTS _agent___, which is instantiated with the `ScoponeGameState` it is facing and shall output the _best_ move, according to its simulated scenario.

We will call it `AgentCarletto` and he has a single purpose: to beat you at Scopone.

### Code

In [60]:

class AgentCarletto(object):
    """
    This class represents a MonteCarlo Agent, which simulates a game-tree starting from
    a given ScoponeGameState.

    It contains methods to:

    - check whether the attributes have been correctly stored.
    - simulate a whole turn with all the other players' move.
    - expand a node by considering all the possible moves and the other players' reactions.
    - perform a complete rollout and backpropagate the corresponding parent move and its reward in the final
    state of the game.
    """

    def __init__(
        self,
        CurrentGameState: ScoponeGameState,
        ComputationalBudget: int = 500
        ) -> None:
        """

        Args:
            CurrentGameState (ScoponeGameState): a ScoponeGameState representing the current game in all its aspects, because
                AgentCarletto needs to know what is doing.
            ComputationalBudget (int, optional): the MCTS algorithm is simulation based; it will explore single branches until a computational budget is exausted.
                Defaults to 100. Higher budgets might lead to a significant slowdown.
        """

        self.CurrentGameState = CurrentGameState
        self.ComputationalBudget = ComputationalBudget

    # def __new__(
    #     cls, *args, **kwargs
    # ):
    #     newagent = "It's A Me, Carletto!"
    #     print(newagent)
    #     return super().__new__(cls)

    def __repr__(self) -> str:
        BadAssIntro = "Hi, my name is Carletto." + " I'm here to kick your ass at Scopone and chew bubble gum." + " And I'm all outta bubble gum."
        return BadAssIntro

    def TurnChecker(
        self,
        CurrentGameState: ScoponeGameState
    ) -> None:
        """
        This function checks that the player starting at a given turn is assigned to the correct team;
        if not, it corrects the assignment.

        Args:
            CurrentGameState (ScoponeGameState): a ScoponeGameState to be checked.
        """
        turn = CurrentGameState.PlayerPosition

        if turn == 0 or turn == 2:
            CurrentGameState.Team = "Hand"
        elif turn == 1 or turn == 3:
            CurrentGameState.Team = "Deck"

    def SimulateTurn(self) -> ScoponeGameState:
        """
        This function simulates all the player's turns. It takes as an input a ScoponeGameState,
        in which the MCTS agent has already made a move.

        Returns:
            ScoponeGameState: a ScoponeGameState resulting from all the players'moves.
        """

        ClonedCurrentGameState = self.CurrentGameState.CloneState()

        if ClonedCurrentGameState.PlayerPosition == 0:
            turns = range(ClonedCurrentGameState.PlayerPosition + 1, ClonedCurrentGameState.PlayerPosition + 4)
        else:
            turns = range(ClonedCurrentGameState.PlayerPosition % 1, ClonedCurrentGameState.PlayerPosition % 4)

        for turn in turns:
            ClonedCurrentGameState.PlayerPosition = turn
            if turn == 0 or turn == 2:
                ClonedCurrentGameState.Team = "Hand"
            elif turn == 1 or turn == 3:
                ClonedCurrentGameState.Team = "Deck"

            ClonedCurrentGameState.Hand = ClonedCurrentGameState.PlayersCards[turn]

            try:
                ClonedCurrentGameState = ClonedCurrentGameState.ApplyMove(
                    ScoponeMove(
                        LegalMoves=ClonedCurrentGameState.Hand,
                        Table=ClonedCurrentGameState.Table,
                    ).GetMove(
                        Greedy=True,
                        Standalone=False
                    )
                )
            except:
                pass

        ClonedCurrentGameState.PlayerPosition = self.CurrentGameState.PlayerPosition
        ClonedCurrentGameState.Team = self.CurrentGameState.Team
        ClonedCurrentGameState.Hand = ClonedCurrentGameState.PlayersCards[ClonedCurrentGameState.PlayerPosition]
        ClonedCurrentGameState.Reward = ClonedCurrentGameState.ComputeRewards()


        return ClonedCurrentGameState

    def Expand(self):
        """
        Expand your CarlettoAgent consciousness and search the Tree.

        This method takes all the possible moves and simulate the consequence of playing them,
        updating the GameState and moving the tree one step further down a given branch.
        It is used as a starter, to evaluate all the possible strategies when the Agent needs to play.
        """

        ClonedCurrentGameState = self.CurrentGameState.CloneState()
        self.TurnChecker(ClonedCurrentGameState)

        #@@@@@@@@@ GENERATING NEW CHILDREN @@@@@@@@@@#

        possible_moves_list = ClonedCurrentGameState.FindChildren()

        resulting_game_states = list()

        for move in possible_moves_list:

            ############ SELECTING NEW CHILDREN ############

            try:
                updated_game_state = AgentCarletto(move).SimulateTurn()
                updated_game_state.ParentMove = move
                resulting_game_states.append(updated_game_state)
            except TypeError:
                EndGame = "We're in the EndGame now.."
                print(EndGame)
                break

        return resulting_game_states

    def Simulate(self) -> tuple:
        """
        This methods performs the rollout (simulate a game until the end)
        and backpropagation (computes the final scores, their rewards and maps them to the origin of the branch,
        and the path).

        Returns:
            tuple:
                - a dict with the simulated game state that started the branch and the reward of the finished game.
                - a list with all the nodes in the sequence, representing the complete branch.
        """

        Simulation = dict()
        TotalReward = 0
        Path = list()
        TablePoints = 0

        ExpandedGameState = self.CurrentGameState.CloneState()
        counter = 0

        while not ExpandedGameState.IsTerminal():
            ExpandedGameState = ExpandedGameState.FindRandomChildren()
            ExpandedGameState = AgentCarletto(ExpandedGameState).SimulateTurn()
            if counter == 0:
                Simulation = ExpandedGameState
                Path.append(ExpandedGameState)
            else:
                TotalReward = ExpandedGameState.Reward
                Path.append(ExpandedGameState)
            counter += 1

        ### This code block is used to compute the reward in the last turn.

        if ExpandedGameState.IsTerminal():
            if isinstance(ExpandedGameState.LastTaker, NAType):
                TakeAllTeam = "Deck"
            elif ExpandedGameState.LastTaker == 0 or ExpandedGameState.LastTaker == 2:
                TakeAllTeam = "Hand"
            elif ExpandedGameState.LastTaker == 1 or ExpandedGameState.LastTaker == 3:
                TakeAllTeam = "Deck"

            for leftover in ExpandedGameState.Table:
                TablePoints += convert_to_card(leftover).Value()

            ExpandedGameState.TeamScores[TakeAllTeam] += TablePoints
            ExpandedGameState.Reward = ExpandedGameState.ComputeRewards()


        return {Simulation:TotalReward}, Path

    def TreeSearch(self) -> dict:
        """
        This method performs the Monte Carlo Tree Search for our Agent and outputs the
        resulting best move.

        Returns:
            dict: a game move {card to be played: card/s to be taken}.
        """

        maxBudget = self.ComputationalBudget

        Parents = self.Expand()
        BestMove = {key: 0 for key in Parents}

        Simulation, Path = self.Simulate()

        while maxBudget > 0:
            NeoSimulation, NeoPath = self.Simulate()
            for State in Simulation.keys():
                for NeoState in NeoSimulation.keys():
                    # No need to explore already seen branches.
                    if Path != NeoPath:
                        if State == NeoState:
                            BestMove[State] = max(Simulation[State], NeoSimulation[NeoState])
                        elif State != NeoState:
                            if Simulation[State] < NeoSimulation[NeoState]:
                                BestMove[NeoState] = NeoSimulation[NeoState]

            maxBudget -= 1

        # points = BestMove[max(BestMove, key=BestMove.get)]
        BestMove = max(BestMove, key=BestMove.get)
        BestMove = BestMove.ParentMove


        return BestMove.ParentMove

---

### Test

Testing whether the MCTS agent is performing as expected is tricky.

On the one hand, it is often impossible (as in really expensive) to analyse and visualize the whole tree, making it a viable option only for scenarios involving the last parts of the game. The slides include a diagram for `test_1`, in which it is easy to recognise which is the optimal move among the possibilities.

On the other hand, the algorithm is __simulation-based__: it is not expected that the __best move__ will be the output, as we are expecting the best move __among the simulated scenarios__, given the __computational budget__. In other words, the tree search does not span the whole tree and there is a random approach to the exploration, as different moves are tried bringing to different rollouts and paths.

Reinforment learning methods come into play in our implementation because we decided to use a __policy__ approach to determine the best option. Given a set of nodes, characterised by an action $a$ (`ParentMove`, the move that generated a node) and the state $s$ deriving from that action, being $R(a)$ the reward associated with that state the decision function can be espressed as:
$$
Q(a, s) = \underbrace{ R(s) }_{\mathrm{Return \ you \ get \ right \ away}}  + \underbrace{ \max_{a'} Q(s', a') }_{\mathrm{Return \ from \ behaving \ optimally \ starting \ from \ state} s'}
$$


Rewards are either negative or positive, to learn the correct strategy among the possibilites.
We are considering two components:

- First, we need to understand whichever are the consequence of the available actions at a given state (playing a card, simulating the other players' strategies).
- Second, we need to choose the best option, given the assumptions that all the players behave optimally after that.

However, this second part would entail to explore _the whole tree_, with a great expense of resources: this is where `FindRandomChild` comes into play, using the Monte Carlo Tree Search algorithm to greatly simplify the search, sparing resources. This means that it is not actually the _optimal_ strategy that we are using to perform the search among the possible game states: we are using a __logical__, __rule-based__ approach (in particular, throught the `Greedy` or `Intermediate` AIs).

The main implication is that __the algorithm is not guaranteed__ to return __the absolute best__, as chance might lead to never explore that path. However, this approach leads to a consistenly realistic adversary, powered by an AI that, while it does not wins _always_, it does not spoil the fun by being an ominscent and unbeatable player.

#### Carletto

All the MCTS methods and data are stored inside the `AgentCarletto` class. To instantiate this AI player, a `ScoponeGameState` is needed, as it contains all the information required.

Note that the MCTS is very information intensive: it is an __omniscent__ player, which knows also every other player's card. This is a necessary step that allows to simulate their reaction to all the possible actions.

In [61]:
Carletto = AgentCarletto(test1_SGS)

In [62]:
Carletto

Hi, my name is Carletto. I'm here to kick your ass at Scopone and chew bubble gum. And I'm all outta bubble gum.

##### `Expand`

The `Expand` method takes the starting node, finds all its possible childrens and returns the resulting game states with their reward $R(a)$.

In other words, the following list of game states represents all the consequences of the possible agent's actions at a given turn.

In [63]:
Carletto.Expand()

[###
 ScoponeGameState:
 ###
 >Parent State: {Card: (1, 2): Card: (1, 1)}.
 > Current hand: [Card: (10, 4), Card: (5, 3)]
 > Current table: [(4, 2), (9, 4), (6, 3)]
 > Points: {'Hand': 542, 'Deck': 500}
 The reward for this State is: 42.
 ###,
 ###
 ScoponeGameState:
 ###
 >Parent State: {Card: (10, 4): [Card: (1, 1), Card: (9, 4)]}.
 > Current hand: [Card: (1, 2), Card: (5, 3)]
 > Current table: [(4, 2), (6, 3)]
 > Points: {'Hand': 546, 'Deck': 500}
 The reward for this State is: 46.
 ###,
 ###
 ScoponeGameState:
 ###
 >Parent State: {Card: (10, 4): [Card: (4, 2), Card: (6, 3)]}.
 > Current hand: [Card: (1, 2), Card: (5, 3)]
 > Current table: [(1, 1), (9, 4)]
 > Points: {'Hand': 542, 'Deck': 500}
 The reward for this State is: 42.
 ###,
 ###
 ScoponeGameState:
 ###
 >Parent State: {Card: (5, 3): [Card: (1, 1), Card: (4, 2)]}.
 > Current hand: [Card: (1, 2), Card: (10, 4)]
 > Current table: [(9, 4), (6, 3)]
 > Points: {'Hand': 555, 'Deck': 500}
 The reward for this State is: 55.
 ###]

##### `SimulateTurn`

This method then advances the game by _simulating the other player's reactions_, under the assumption that everyone behaves optimally; in our case, all the other player are simulated by using rule-based AI algorithms.

In [64]:
Carletto.SimulateTurn()

###
ScoponeGameState:
###
>Parent Move: <NA>.
> Current hand: [(1, 2), (10, 4), (5, 3)]
> Current table: []
> Points: {'Hand': 542, 'Deck': 1691}
The reward for this State is: -1149.
###

In this case, the example makes it clear why not thinking ahead, as the other functions do, can be a major source of headaches in a game. `Carletto`'s move allowed a _scopa_ for the other team.

##### `Simulate`

This is the method that performs both the __rollout__. Building upon the _expanded_ original game scenario and all the other players' reactions, it fully explores a _branch_ by __randomly selecting__ a possible move at each subsequent node, while at the same time keeping track of the _score_.

In [65]:
Carletto.Simulate()

({###
  ScoponeGameState:
  ###
  >Parent Move: {Card: (1, 2): Card: (1, 1)}.
  > Current hand: [(10, 4), (5, 3)]
  > Current table: [(6, 3), (10, 3)]
  > Points: {'Hand': 570, 'Deck': 520}
  The reward for this State is: 50.
  ###: -140},
 [###
  ScoponeGameState:
  ###
  >Parent Move: {Card: (1, 2): Card: (1, 1)}.
  > Current hand: [(10, 4), (5, 3)]
  > Current table: [(6, 3), (10, 3)]
  > Points: {'Hand': 570, 'Deck': 520}
  The reward for this State is: 50.
  ###,
  ###
  ScoponeGameState:
  ###
  >Parent Move: {Card: (5, 3): []}.
  > Current hand: [(10, 4)]
  > Current table: [(6, 3), (5, 3), (8, 3), (9, 2)]
  > Points: {'Hand': 570, 'Deck': 669}
  The reward for this State is: -99.
  ###,
  ###
  ScoponeGameState:
  ###
  >Parent Move: {Card: (10, 4): []}.
  > Current hand: []
  > Current table: [(6, 3), (5, 3), (8, 3), (9, 2), (10, 4)]
  > Points: {'Hand': 633, 'Deck': 710}
  The reward for this State is: -77.
  ###])

This succession of game states is therefore a __fully developed branch__, in which one among the expanded _children_ of the initial game state is developed until the game is finished, creating a `Path` of linked `ScoponeGameStates`.

Note that for each game state $s$ the algorithm computes $a$ (`ParentMove`) and $R(s)$ (`Reward`).

##### `TreeSearch`

All the previous methods are then combined in the `TreeSearch`: until a given `ComputationalBudget` is depleted, games are simulated. The original move is stored and brought forward, until a terminal state is not reached (there are no more cards to be played, all hands are empty).

Then, both this _original move_ and the _reward_ achieved at the end of the game are _propagated backward_ and stored; new simulations are made and at every simulation their reward are compared, with only the best choice kept as the optimal move.

This implements the second part of the fundamental computation that allows the AI to __learn__ what is the optimal move through an iterative search of the tree with the Monte Carlo method, and this move will be the output of this method, which is the only one that needs to be called to use the algorithm in a game.

In [66]:
Carletto.TreeSearch()

{Card: (1, 2): Card: (1, 1)}

The performance, at a computational budget of 500 (number of simulation runs), makes it more than usable inside a program.

In [67]:
%timeit Carletto.TreeSearch()

1.04 s ± 8.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Carletto III

In [68]:
AgentCarlettoTheThird = AgentCarletto(test3_SGS)

In [69]:
AgentCarlettoTheThird.TreeSearch()

{Card: (3, 3): []}

##### BigCarletto

In this scenario, we are considering a game that has just started, to compare the performance with a tree of maximum extension.

In [70]:
ft_PlayerPosition = 0
ft_PlayerCards = {
    0:
    [
        (8, 2),
        (8, 4),
        (5, 2),
        (10, 3),
        (7, 4),
        (4, 1),
        (1, 2),
        (10, 1),
        (9, 1),
        (6, 2)
    ],
    1:
    [
        (1, 3),
        (4, 4),
        (8, 3),
        (7, 1),
        (6, 3),
        (7, 3),
        (1, 4),
        (2, 3),
        (4, 2),
        (2, 2)
    ],
    2:
    [
        (9, 2),
        (10, 2),
        (6, 1),
        (10, 4),
        (5, 4),
        (3, 4),
        (2, 4),
        (3, 3),
        (9, 4),
        (8, 1)
    ],
    3:
    [
        (1, 1),
        (2, 1),
        (3, 1),
        (7, 2),
        (5, 3),
        (3, 2),
        (9, 3),
        (4, 3),
        (5, 1),
        (6, 4)
    ]
}
ft_Team = "Hand"
ft_TeamScores = {
    "Hand": 0,
    "Deck": 0
}

In [71]:
final_test = ScoponeGameState(
    ft_PlayerPosition,
    ft_PlayerCards,
    ft_Team,
    ft_TeamScores
)

In [72]:
BigCarletto = AgentCarletto(final_test)

In [73]:
BigCarletto.TreeSearch()

{Card: (4, 1): []}

In [74]:
%timeit BigCarletto.TreeSearch()

5.63 s ± 31.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As a concluding remark, there is an important trade-off here to be considered between __computational resources__ and __performance__. The probability of reaching the absolute optimum and therefore returning the absolute best move is increasing in the number of simulations; however, this could potentially slow down the performance excessively while being only partially useful: as the game progresses, the possible paths shrink, making a high number of simulations much less useful to reach a good solution. Moreover, all the game needs to be simulated again __at every turn__, because the assumption that everyone plays optimally or actually follows a `Greedy` strategy is only useful as a simulation technique, while the algorithm needs to react to the actual scenario generated after it has returned a move to be effective, waiting for the other players; this means that a __full exploration of the tree__ is not a reasonable strategy, as demonstrated by the worse performance of `MinMax` algorithms in a game setting (Di Palma, 2014).

The __simulation-based Monte Carlo__, mixed with a __reinforcement learning approach to move selection__, yields a intrinsically random output that actually allows for a better AI player, more challenging than rule-based AI while not being unbeatable and, therefore, not fun.